In [79]:
import time
from vina import Vina

t = time.time()

path = '../../vina'

v = Vina(sf_name='vina')

v.set_receptor(f'{path}/5r84.pdbqt')

v.set_ligand_from_file(f'{path}/1iep_ligand.pdbqt')

v.compute_vina_maps(center=[9.812, -0.257, 20.8485], box_size=[14.328, 8.85, 12.539])
v.dock(exhaustiveness=8, n_poses=1)

# Score the current pose
energy = v.score()
print('Score before minimization: %.3f (kcal/mol)' % energy[0])

# Minimized locally the current pose
energy_minimized = v.optimize()
# print('Score after minimization : %.3f (kcal/mol)' % energy_minimized[0])
# v.write_pose('1iep_ligand_minimized.pdbqt', overwrite=True)

# # Dock the ligand
v.dock(exhaustiveness=8, n_poses=1)

print(t - time.time())

Computing Vina grid ... Score before minimization: 2.149 (kcal/mol)
-132.23124432563782
done.
Performing docking (random seed: 948702018) ... 
0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

mode |   affinity | dist from best mode
     | (kcal/mol) | rmsd l.b.| rmsd u.b.
-----+------------+----------+----------
   1        2.149          0          0
Performing local search ... done.
Performing docking (random seed: 948702018) ... 
0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************


In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem
import subprocess

# Load molecules
protein = Chem.MolFromPDBFile('protein.pdb')
original_ligand = Chem.MolFromPDBFile('original_ligand.pdb')
new_ligand = Chem.MolFromMolFile('new_ligand.mol')

# Align new ligand to original ligand
AllChem.AlignMol(new_ligand, original_ligand)

In [77]:
# %%time

# from vina import Vina

# path = '../../vina'

# v = Vina(sf_name='vina')

# v.set_receptor(f'{path}/1iep_receptor.pdbqt')

# v.set_ligand_from_file(f'{path}/1iep_ligand.pdbqt')
# v.compute_vina_maps(center=[15.190, 53.903, 16.917], box_size=[20, 20, 20])

# # Score the current pose
# energy = v.score()
# energy_minimized = v.optimize()

In [55]:
from rdkit import Chem
from rdkit.Chem import AllChem
from openbabel import openbabel
import io

def smiles_to_pdbqt(smiles):
    
    # Convert SMILES to RDKit molecule
    mol = Chem.MolFromSmiles(smiles)
    if not mol:
        raise ValueError("Invalid SMILES string")
    
    # Generate 3D coordinates
    mol = Chem.AddHs(mol)
    AllChem.EmbedMolecule(mol, AllChem.ETKDG())
    AllChem.MMFFOptimizeMolecule(mol)
    
    # Compute Gasteiger charges
    AllChem.ComputeGasteigerCharges(mol)

    # Convert RDKit molecule to SDF format in memory
    sdf_stream = io.StringIO()
    writer = Chem.SDWriter(sdf_stream)
    writer.write(mol)
    writer.close()

    # Convert in-memory SDF to Open Babel molecule
    obConversion = openbabel.OBConversion()
    obConversion.SetInAndOutFormats("sdf", "pdbqt")
    mol_ob = openbabel.OBMol()
    obConversion.ReadString(mol_ob, sdf_stream.getvalue())
    
    # Update Open Babel molecule with Gasteiger charges from RDKit molecule
    for atom, atom_ob in zip(mol.GetAtoms(), openbabel.OBMolAtomIter(mol_ob)):
        charge = atom.GetProp("_GasteigerCharge")
        atom_ob.SetPartialCharge(float(charge))

    pdbqt_data = obConversion.WriteString(mol_ob)
    
    return pdbqt_data

# Test
smiles_string = "CCO"  # Ethanol
pdbqt_data = smiles_to_pdbqt(smiles_string)
print(pdbqt_data)

REMARK  Name = 
REMARK  2 active torsions:
REMARK  status: ('A' for Active; 'I' for Inactive)
REMARK    1  A    between atoms: C_1  and  C_2
REMARK    2  A    between atoms: C_2  and  O_3
REMARK                            x       y       z     vdW  Elec       q    Type
REMARK                         _______ _______ _______ _____ _____    ______ ____
ROOT
ATOM      1  C   UNL     1      -0.888  -0.166   0.035  0.00  0.00    +0.034 C 
ATOM      2  C   UNL     1       0.488   0.436   0.226  0.00  0.00    +0.152 C 
ENDROOT
BRANCH   2   3
ATOM      3  O   UNL     1       1.486  -0.534  -0.057  0.00  0.00    -0.397 OA
ATOM      4  H   UNL     1       1.366  -0.817  -0.980  0.00  0.00    +0.210 HD
ENDBRANCH   2   3
TORSDOF 1



In [56]:
%load_ext autoreload
%autoreload 2

import rdkit
rdkit.Chem.Draw.IPythonConsole.ipython_maxProperties = -1

import dgym as dg

# load all data
print('load data')
path = '../../dgym-data'

deck = dg.MoleculeCollection.from_sdf(
    f'{path}/DSi-Poised_Library_annotated.sdf',
    reactant_names=['reagsmi1', 'reagsmi2', 'reagsmi3']
)

load data


In [63]:
import meeko
import rdkit

paxlovid = 'CC1(C2C1C(N(C2)C(=O)C(C(C)(C)C)NC(=O)C(F)(F)F)C(=O)NC(CC3CCNC3=O)C#N)C'

# load ligand from SMILES into RDKit
lig = rdkit.Chem.MolFromSmiles(paxlovid)

# add hydrogens (without regard to pH)
protonated_lig = rdkit.Chem.AddHs(lig)

# generate 3D coordinates for the ligand. 
rdkit.Chem.AllChem.EmbedMolecule(protonated_lig)

# convert to a PDBQT string
meeko_prep = meeko.MoleculePreparation()
mol_setups = meeko_prep.prepare(protonated_lig)
lig_pdbqt = meeko.PDBQTWriterLegacy.write_string(mol_setups[0])

In [67]:
meeko.MoleculePreparation??

Init signature:
meeko.MoleculePreparation(
    merge_these_atom_types=('H',),
    hydrate=False,
    flexible_amides=False,
    rigid_macrocycles=False,
    min_ring_size=7,
    max_ring_size=33,
    keep_chorded_rings=False,
    keep_equivalent_rings=False,
    double_bond_penalty=50,
    rigidify_bonds_smarts=[],
    rigidify_bonds_indices=[],
    atom_type_smarts={},
    add_atom_types=[],
    reactive_smarts=None,
    reactive_smarts_idx=None,
    add_index_map=False,
    remove_smiles=False,
)
Docstring:      <no docstring>
Source:        
class MoleculePreparation:
    def __init__(self,
            merge_these_atom_types=("H",),
            hydrate=False,
            flexible_amides=False,
            rigid_macrocycles=False,
            min_ring_size=7,
            max_ring_size=33,
            keep_chorded_rings=False,
            keep_equivalent_rings=False,
            double_bond_penalty=50,
            rigidify_bonds_smarts=[],
            rigidify_bonds_indices=[],
      

In [64]:
print(lig_pdbqt[0])

REMARK SMILES CC(C)(C)C(NC(=O)C(F)(F)F)C(=O)N1CC2C(C1C(=O)NC(C#N)CC1CCNC1=O)C2(C)C
REMARK SMILES IDX 34 1 33 2 17 3 18 4 19 5 15 6 16 7 35 8 20 9 21 10 22 11
REMARK SMILES IDX 23 13 26 14 27 15 28 16 31 17 29 18 30 19 32 20 24 22 25 23
REMARK SMILES IDX 13 24 14 25 5 26 2 27 1 28 3 29 4 30 6 31 7 32 8 34 9 35
REMARK SMILES IDX 10 36 11 37 12 38
REMARK H PARENT 22 12 30 21 6 33
REMARK Flexibility Score: inf
ROOT
ATOM      1  C   UNL     1       0.490   5.230  -1.728  1.00  0.00     0.012 C 
ATOM      2  C   UNL     1       0.458   3.706  -1.483  1.00  0.00    -0.025 C 
ATOM      3  C   UNL     1      -0.677   3.058  -2.193  1.00  0.00     0.021 C 
ATOM      4  C   UNL     1      -0.636   3.219  -0.674  1.00  0.00     0.033 C 
ATOM      5  C   UNL     1      -0.863   1.825  -0.185  1.00  0.00     0.178 C 
ATOM      6  N   UNL     1      -1.298   1.043  -1.307  1.00  0.00    -0.328 N 
ATOM      7  C   UNL     1      -0.661   1.588  -2.485  1.00  0.00     0.125 C 
ATOM      8  C   UNL     

In [76]:
setups = meeko_prep.prepare(protonated_lig)

In [79]:
print(meeko_prep.write_pdbqt_string(setups[0]))

REMARK SMILES CC(C)(C)C(NC(=O)C(F)(F)F)C(=O)N1CC2C(C1C(=O)NC(C#N)CC1CCNC1=O)C2(C)C
REMARK SMILES IDX 34 1 33 2 17 3 18 4 19 5 15 6 16 7 35 8 20 9 21 10 22 11
REMARK SMILES IDX 23 13 26 14 27 15 28 16 31 17 29 18 30 19 32 20 24 22 25 23
REMARK SMILES IDX 13 24 14 25 5 26 2 27 1 28 3 29 4 30 6 31 7 32 8 34 9 35
REMARK SMILES IDX 10 36 11 37 12 38
REMARK H PARENT 22 12 30 21 6 33
REMARK Flexibility Score: inf
ROOT
ATOM      1  C   UNL     1      -3.616   3.204   1.813  1.00  0.00     0.012 C 
ATOM      2  C   UNL     1      -2.755   2.905   0.569  1.00  0.00    -0.025 C 
ATOM      3  C   UNL     1      -2.934   1.622  -0.008  1.00  0.00     0.021 C 
ATOM      4  C   UNL     1      -1.637   2.077   0.734  1.00  0.00     0.033 C 
ATOM      5  C   UNL     1      -0.826   1.190  -0.217  1.00  0.00     0.178 C 
ATOM      6  N   UNL     1      -1.338  -0.042   0.209  1.00  0.00    -0.328 N 
ATOM      7  C   UNL     1      -2.847   0.234   0.420  1.00  0.00     0.125 C 
ATOM      8  C   UNL     